# Build Your AI Agent with MCPs using SGLang, Pydantic AI, and AMD MI300X GPU

Welcome to this hands-on workshop! Throughout this tutorial, we'll leverage AMD GPUs and **Model Context Protocol (MCP)** ,an open standard for exposing LLM tools via API, to deploy powerful language models like Qwen2.5-7B. Key components:
- 🖥️ **SGLang** for GPU-optimized inference
- 🛠️ **Pydantic-AI** for agent/tool management
- 🔌 **MCP Servers** for pre-built tool integration

You'll learn how to set up your environment, deploy large language models like Qwen2.5, connect them to real-world tools using MCP, and build a conversational agent capable of reasoning and taking actions.

By the end of this workshop, you’ll have built an AI-powered assistant agent that can find a place to stay based on your preferences like location, budget, and travel dates.

Let’s dive in!

## Table of Contents

- [Step 1: Launching SGLang Server on AMD GPUs](#step1)
- [Step 2: Installing Dependencies](#step2)
- [Step 3: Create a simple instance of Pydantic-AI Agent](#step3)
- [Step 4: Write a Date/Time Tool for your Agent](#step4)
- [Step 5: Replace your Date/Time Tool with a MCP Server](#step5)
- [Step 6: Turn your Agent into a trip planner](#step6)
- [Step 7: Challenge](#step7)

<a id="step1"></a>

## Step 1: Launch a SGLang Server

In this workshop we are going to use [SGLang](https://github.com/sgl-project/sglang) as our inference serving engine. SGLang provides many benefits such as fast model execution, extensive list of supported models, easy to use, and best of all it's open-source. 

### Deploy Qwen2.5-7B-Instruct Model with SGLang (~3 mins)




First, we need to start a SGLang server and create an OpenAI-compatible endpoint for your LLM:

In [ ]:
import os
from sglang.test.doc_patch import launch_server_cmd
from sglang.utils import wait_for_server

os.environ["SGLANG_USE_AITER"] = "1"
_server_process, _port = launch_server_cmd(
    "python3 -m sglang.launch_server --model-path Qwen/Qwen2.5-7B-Instruct --tool-call-parser qwen25 --host 0.0.0.0 --port 8000"  # qwen25
)
wait_for_server(f"http://localhost:8000")

/usr/local/lib/python3.12/dist-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm
/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


/usr/local/lib/python3.12/dist-packages/transformers/utils/hub.py:111: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(
INFO 08-15 16:03:47 __init__.py:179] Automatically detected platform rocm.
WARNING 08-15 16:03:47 rocm.py:34] `fork` method is not supported by ROCm. VLLM_WORKER_MULTIPROC_METHOD is overridden to `spawn` instead.
/sgl-workspace/hubert_sgl/python/sglang/srt/layers/quantization/awq.py:42: UserWarning: Using kernels directly from vllm. This might lead to performance degradation or missing functionalities as certain kernels may not be optimized. 
  warnings.warn(
/sgl-workspace/hubert_sgl/python/sglang/srt/layers/quantization/awq.py:60: UserWarning: HIP does not support fused_marlin_moe currently.
  warnings.warn(f"HIP does not support fused_marlin_moe currently.")
[2025-08-15 16:03:49] server_args=ServerArgs(model_path='Qwen/Qwen2.5-7B-Instruct', tokenizer_path='Qwen/Qwen2.5-7B-Inst

You can open a terminal and watch the GPU utilization by running this command:

```bash
watch rocm-smi
```

Let's set some environment variables for our server to use throughout this tutorial:

In [ ]:
BASE_URL = f"http://localhost:8000/v1"

os.environ["BASE_URL"]    = BASE_URL
os.environ["OPENAI_API_KEY"] = "abc-123"   

print("Config set:", BASE_URL)

Config set: http://localhost:30221/v1


We can verify your model is available at the `BASE_URL` we just set by running the following command.

In [ ]:
!curl http://localhost:8000/v1/models -H "Authorization: Bearer $OPENAI_API_KEY"

curl: (3) URL using bad/illegal format or missing URL


Congratulations, you now just launched a powerful server that can serve any incoming request and allowing you to build amazing applications. Wasn't that easy?🎉 

<a id="step2"></a>

## Step 2: Installing Dependencies

We are going to use `Pydantic AI`. Let's install the dependencies:

In [4]:
import sys, subprocess
def pip_install(pkgs):
    subprocess.check_call([sys.executable, "-m", "pip", "install", "-q", *pkgs])

need = []
for mod in ("mcp", "httpx", "duckduckgo_search", "pydantic_ai"):
    try:
        __import__(mod if mod != "duckduckgo_search" else "duckduckgo_search")
    except Exception:
        need.append(mod)

if need:
    pip_install(need)

print("OK: deps present.")

OK: deps present.



<a id="step3"></a>

## Step 3: Create a simple instance of Pydantic-AI Agent

Let's start by creating a custom OpenAI Compatible endpoint for our agent. 


In [5]:
from pydantic_ai.models.openai import OpenAIModel
from pydantic_ai.providers.openai import OpenAIProvider

provider = OpenAIProvider(
    base_url=os.environ["BASE_URL"],
    api_key=os.environ["OPENAI_API_KEY"],
)

agent_model = OpenAIModel("Qwen2.5-7B-Instruct", provider=provider)

Let's start by creating an instance the `Agent` class from `pydantic_ai`. 


In [6]:
from pydantic_ai import Agent

agent = Agent(
    model=agent_model
)

It's time to test the agent. `pydantic_ai` provides multiple ways to run `Agent`. You can learn more about it [here](https://ai.pydantic.dev/agents/#running-agents).

In this workshop, we are running in `async` mode. We are going to define a helper function that allows us to quickly test our agent throughout this workshop.

In [7]:
import asyncio
from pydantic_ai.mcp import MCPServerStdio
async def run_async(prompt: str) -> str:
    async with agent:
        result = await agent.run(prompt)
        return result.output


Test the agent by calling this function.

In [8]:
await run_async("What is the capital of France?")

[2025-08-15 16:04:22] Prefill batch. #new-seq: 1, #new-token: 36, #cached-token: 0, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-15 16:04:22] INFO:     127.0.0.1:43958 - "POST /v1/chat/completions HTTP/1.1" 200 OK


'The capital of France is Paris.'

Great! now that we have the basics of creating an agent instance, and connecting it to the model we started serving with vLLM earlier.

<a id="step4"></a>

## Step 4: Write a Date/Time Tool for Your Agent

LLMs naturally rely on their training data to respond to your prompts. Therefore, the agent we just defined fails to answer a factual question that falls outside of it's training knowledge. Let's show this with an example:

In [9]:
await run_async("What’s the date today?")

[2025-08-15 16:04:22] Prefill batch. #new-seq: 1, #new-token: 10, #cached-token: 25, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-15 16:04:22] Decode batch. #running-req: 1, #token: 60, token usage: 0.00, cuda graph: True, gen throughput (token/s): 5.61, #queue-req: 0, 
[2025-08-15 16:04:22] INFO:     127.0.0.1:43958 - "POST /v1/chat/completions HTTP/1.1" 200 OK


"As an AI, I don't have real-time capabilities and I don't have access to current dates. If you need to know today's date, I recommend checking a calendar or an online source for the most accurate information."

It is no surprise that the model failed to answer this question. Now, it's time to power-up your LLM by providing `agent` a function that can get the current date. The process of an LLM triggering a function call is commonly referred to as `Tool Calling` or `Function Calling`. In this workshop we are going to take advantage of `pydantic-ai`'s agent `tools` to provide our agent appropriate tools. First, we need to define a custom tool. Below is how we can define a tool in this framework.

In [10]:
from datetime import datetime
from pydantic_ai import Tool          
@Tool
def get_current_date() -> str:
    """Return the current date/time as an ISO-formatted string."""
    return datetime.now().strftime("%Y-%m-%d %H:%M:%S")

Next, we need to provide this tool to our Agent, as this will notify the LLM about the existence of such a tool we just definied. This is simply done by just providing the function signiture of the tool we just defined to our agent constructor. 

In [11]:
agent = Agent(
    model=agent_model,
    tools=[get_current_date],
    system_prompt = (
        "You have access to:\n"
        "   1. get_current_time(params: dict)\n"
        "Use this tool for date/time questions."
    )
)

Let's test the agent.

In [12]:
await run_async("What’s the date today?")

[2025-08-15 16:04:22] Prefill batch. #new-seq: 1, #new-token: 164, #cached-token: 4, token usage: 0.00, #running-req: 0, #queue-req: 0, 


[2025-08-15 16:04:22] INFO:     127.0.0.1:43958 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-08-15 16:04:22] Prefill batch. #new-seq: 1, #new-token: 41, #cached-token: 181, token usage: 0.01, #running-req: 0, #queue-req: 0, 
[2025-08-15 16:04:22] Decode batch. #running-req: 1, #token: 224, token usage: 0.01, cuda graph: True, gen throughput (token/s): 151.87, #queue-req: 0, 
[2025-08-15 16:04:22] INFO:     127.0.0.1:43958 - "POST /v1/chat/completions HTTP/1.1" 200 OK


"Today's date is 2025-08-15."

Well done on building an agent with access to real-time data. 



<a id="step5"></a>

## Step 5: Replace Your Date/time Tool with a MCP server

Now that we learned how to create a custom tool and provide the agent access to this tool. Let's now explore a trendy topic of [Model Context Protocol](https://modelcontextprotocol.io/introduction). We are going to explore how we can replace our custom tool with a simple MCP server that can serve our agent and provide similar information.

**Why MCP?** MCP servers provide:
- ✅ Standardized API interfaces
- 🔄 Reusable across projects
- 📦 Pre-built functionality

Let's replace our custom time tool with an official MCP time server:

### Installing Time MCP Server

We are going to start by installing this MCP server:


In [13]:
!pip install -q mcp-server-time

Now let's define our time_server:

In [14]:
from pydantic_ai.mcp import MCPServerStdio

time_server = MCPServerStdio(
    "python",
    args=[
        "-m", "mcp_server_time",
        "--local-timezone=America/New_York",
    ],
)

Finally, let's modify our agent to remove our previously defined tool, and add this MCP server instead.

In [15]:
agent = Agent(
    model=agent_model,
    mcp_servers=[time_server],
    system_prompt = (
        "You are a helpful agent and you have access to this tool:\n"
        "   get_current_time(params: dict)\n"
        "When the user asks for the current date or time, call get_current_time.\n"
    )
)

Great, let's see if the agent can use the MCP to give us the correct time now.

In [16]:
await run_async("What’s the date today?")

[2025-08-15 16:06:11] Prefill batch. #new-seq: 1, #new-token: 423, #cached-token: 5, token usage: 0.00, #running-req: 0, #queue-req: 0, 
[2025-08-15 16:06:11] INFO:     127.0.0.1:41416 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-08-15 16:06:11] Decode batch. #running-req: 1, #token: 0, token usage: 0.00, cuda graph: True, gen throughput (token/s): 0.37, #queue-req: 0, 
[2025-08-15 16:06:11] Prefill batch. #new-seq: 1, #new-token: 79, #cached-token: 433, token usage: 0.02, #running-req: 0, #queue-req: 0, 
[2025-08-15 16:06:11] INFO:     127.0.0.1:41416 - "POST /v1/chat/completions HTTP/1.1" 200 OK


"Today's date in New York is August 15, 2025."

# Step 6: Turn your agents for travel planning


In [17]:
!curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash -
!sudo apt-get install -y nodejs
!node -v && npm -v && npx -v

2025-08-15 16:06:17 - Installing pre-requisites
Hit:1 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:2 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]      
Get:3 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]        
Get:4 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [127 kB]      
Hit:5 https://repo.radeon.com/amdgpu/6.3/ubuntu jammy InRelease                
Get:6 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [3521 kB]
Hit:7 https://repo.radeon.com/rocm/apt/6.3 jammy InRelease                     
Get:8 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 Packages [1575 kB]
Get:9 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease [18.1 kB]
Get:10 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy/main amd64 Packages [33.2 kB]
Fetched 5531 kB in 1s (5537 kB/s)    
Reading package lists... Done
W: https://repo.radeon.com/amdgpu/6.3/ubuntu/dists/jammy/InRelease: Key is

---


Tadaa! Now you have officially used MCP servers to power-up your AI agents. In the next section we show how you can your turn many ideas into real working projects by using free MCP servers available today.



<a id="step6"></a>

## Step 6: Turn your agent to a travel planner

As we experience in the last section, MCP servers are really easy to use and they provide a standard way of providing LLMs the tools we need. There are already thousands of MCP servers available for us to use. There are some MCP trackers that you can always use to find out about available servers. Here are some for your reference:
- https://github.com/modelcontextprotocol/servers
- https://mcp.so/

We are going to use npx to launch out next server. Therefore, let's install the required dependencies.

In [ ]:
# Install Node.js 20 via NodeSource
!curl -fsSL https://deb.nodesource.com/setup_20.x | sudo -E bash -
!apt install -y nodejs

Verify `npm` and `npx` installation:

In [ ]:
!node -v && npm -v && npx --version

In [ ]:
import sys, os, subprocess, pkgutil, pathlib, textwrap, asyncio, re, json, math
from typing import Any, Dict, List, Optional
from datetime import datetime, timedelta

# 0) Basics + ensure deps
PY = sys.executable
ROOT = pathlib.Path.cwd() / "mcp_nb"
ROOT.mkdir(exist_ok=True)
BASE_ENV = os.environ.copy()
BASE_ENV["PYTHONUNBUFFERED"] = "1"

def ensure_pkgs():
    to_check = [
        ("mcp", "mcp"),
        ("pydantic_ai", "pydantic-ai"),
        ("duckduckgo_search", "duckduckgo-search"),
        ("httpx", "httpx"),
        ("tzdata", "tzdata"),
    ]
    missing = []
    for mod, pipname in to_check:
        if not any(m == mod or m.startswith(mod + ".") for m in sys.modules):
            if not pkgutil.find_loader(mod):
                missing.append(pipname)
    if missing:
        print("Installing:", ", ".join(sorted(set(missing))))
        subprocess.check_call([PY, "-m", "pip", "install", "-q", *sorted(set(missing))])

ensure_pkgs()

/tmp/ipykernel_786/1479450536.py:26: DeprecationWarning: 'pkgutil.find_loader' is deprecated and slated for removal in Python 3.14; use importlib.util.find_spec() instead
  if not pkgutil.find_loader(mod):


---

In [ ]:
import sys, os, pkgutil, subprocess, pathlib, textwrap, asyncio, re, json, math
from typing import Any, Dict, List, Optional, Tuple
from datetime import datetime, timedelta

# 0) Basics + ensure deps
PY = sys.executable
ROOT = pathlib.Path.cwd() / "mcp_nb"
ROOT.mkdir(exist_ok=True)
BASE_ENV = os.environ.copy()
BASE_ENV["PYTHONUNBUFFERED"] = "1"

def ensure_pkgs():
    to_check = [
        ("mcp", "mcp"),
        ("pydantic_ai", "pydantic-ai"),
        ("duckduckgo_search", "duckduckgo-search"),
        ("httpx", "httpx"),
        ("tzdata", "tzdata"),
        ("openai", "openai"),          # for OpenAIModel
        ("anthropic", "anthropic"),    # optional AnthropicModel
        ("nest_asyncio", "nest-asyncio"),
    ]
    missing = []
    for mod, pipname in to_check:
        if not any(m == mod or m.startswith(mod + ".") for m in sys.modules):
            if not pkgutil.find_loader(mod):
                missing.append(pipname)
    if missing:
        print("Installing:", ", ".join(sorted(set(missing))))
        subprocess.check_call([PY, "-m", "pip", "install", "-q", *sorted(set(missing))])

ensure_pkgs()

import nest_asyncio
nest_asyncio.apply()

# 1) Local MCP servers (Search / Time)
search_mcp_code = '''\
from typing import List, Dict
import re
from mcp.server.fastmcp import FastMCP

mcp = FastMCP("SearchMCP")

try:
    from duckduckgo_search import DDGS
except Exception:
    DDGS = None

@mcp.tool()
def web_search(query: str, max_results: int = 5) -> List[Dict]:
    """DuckDuckGo web search. Returns [{title, href, snippet}]"""
    out = []
    if DDGS is None:
        return [{"title":"[search unavailable]","href":"","snippet":"duckduckgo_search not installed or failed to import."}]
    try:
        with DDGS() as ddg:
            for r in ddg.text(query, max_results=max(1, min(int(max_results), 10))):
                out.append({"title": r.get("title",""), "href": r.get("href",""), "snippet": r.get("body","")})
    except Exception as e:
        out.append({"title":"[search error]", "href":"", "snippet":f"{type(e).__name__}: {e}"})
    return out

@mcp.tool()
async def fetch(url: str, max_chars: int = 2000) -> Dict:
    """Fetch a URL and return {url, status, text[:max_chars]}"""
    import httpx, re
    try:
        async with httpx.AsyncClient(follow_redirects=True, timeout=20) as client:
            resp = await client.get(url)
            text = re.sub(r"\\s+", " ", resp.text)
            return {"url": str(resp.url), "status": resp.status_code, "text": text[:max_chars]}
    except Exception as e:
        return {"url": url, "status": 0, "text": f"{type(e).__name__}: {e}"}

if __name__ == "__main__":
    mcp.run("stdio")
'''

time_mcp_code = '''\
from mcp.server.fastmcp import FastMCP
from datetime import datetime
from typing import Dict, Any
try:
    from zoneinfo import ZoneInfo
except Exception:
    ZoneInfo = None

mcp = FastMCP("TimeMCP")

@mcp.tool()
def now(tz: str = "UTC") -> Dict[str, Any]:
    try:
        z = ZoneInfo(tz) if ZoneInfo else None
    except Exception:
        z = None
    if z is None:
        from datetime import timezone
        z = timezone.utc
        tz = "UTC"
    dt = datetime.now(z)
    return {"iso": dt.isoformat(), "date": dt.strftime("%Y-%m-%d"), "time": dt.strftime("%H:%M:%S"), "tz": str(z)}

if __name__ == "__main__":
    mcp.run("stdio")
'''

ROOT.joinpath("search_mcp.py").write_text(textwrap.dedent(search_mcp_code), encoding="utf-8")
ROOT.joinpath("time_mcp.py").write_text(textwrap.dedent(time_mcp_code), encoding="utf-8")
print("Wrote servers to", ROOT)

# 2) Launch MCP servers
from pydantic_ai.mcp import MCPServerStdio

def mk_stdio(cmd, args, name):
    return MCPServerStdio(
        cmd, args=args,
        env=BASE_ENV, cwd=str(ROOT),
        timeout=180,
        log_level="warn",
        log_handler=lambda e: None,   # set to print(...) for verbose logs
    )

time_server    = mk_stdio(sys.executable, [str(ROOT / "time_mcp.py")], "time")
search_server  = mk_stdio(sys.executable, [str(ROOT / "search_mcp.py")], "search")

async def probe(name, server):
    try:
        tools = await server.list_tools()
        print(f"✓ {name} ready: {', '.join(t.name for t in tools)}")
        return server
    except Exception as e:
        print(f"✗ {name} disabled: {e}")
        return None

servers = asyncio.run(asyncio.gather(
    probe("time", time_server),
    probe("search", search_server),
))
servers = [s for s in servers if s]

# 3) Models & helpers
from pydantic import BaseModel, Field
from IPython.display import Markdown, display

class Link(BaseModel):
    title: str
    url: str = ""
    price: Optional[float] = None
    notes: Optional[str] = None

class DayPlan(BaseModel):
    date: str
    summary: str = ""
    activities: List[str] = Field(default_factory=list)

class CostSummary(BaseModel):
    currency: str = "USD"
    flights: float = 0.0
    hotels: float = 0.0
    activities: float = 0.0
    transit: float = 0.0
    total: float = 0.0

class TripPlan(BaseModel):
    flights: List[Link] = Field(default_factory=list)
    hotels: List[Link] = Field(default_factory=list)
    day_by_day: List[DayPlan] = Field(default_factory=list)
    transit_notes: str = ""
    links: List[Link] = Field(default_factory=list)
    cost_summary: CostSummary = Field(default_factory=CostSummary)

class PlanSkeleton(BaseModel):
    destination_city: str
    start_date: str
    end_date: str
    day_by_day: List[DayPlan] = Field(default_factory=list)
    notes: str = ""

class DayIdeas(BaseModel):
    date: str
    ideas: List[Link] = Field(default_factory=list)

class ResearchOutput(BaseModel):
    days: List[DayIdeas] = Field(default_factory=list)
    links: List[Link] = Field(default_factory=list)

# 4) LLM provider pick
MODEL = OpenAIModel("Qwen2.5-7B-Instruct", provider=provider)

# 5) Utilities (dates, links)
def _valid_ymd(s: Optional[str]) -> bool:
    if not isinstance(s, str): return False
    try:
        datetime.strptime(s.strip(), "%Y-%m-%d")
        return True
    except Exception:
        return False

def daterange(start_date: str, end_date: str) -> List[str]:
    s = datetime.strptime(start_date, "%Y-%m-%d"); e = datetime.strptime(end_date, "%Y-%m-%d")
    out, d = [], s
    while d <= e:
        out.append(d.strftime("%Y-%m-%d")); d += timedelta(days=1)
    return out

from urllib.parse import quote
def google_flights_link(origin: str, dest: str, depart: str, ret: str) -> str:
    q = f"Flights to {dest} from {origin} on {depart} through {ret}"
    return "https://www.google.com/travel/flights/search?q=" + quote(q)

def booking_link(city: str, start: str, end: str) -> str:
    return f"https://www.booking.com/searchresults.html?ss={quote(city)}&checkin={start}&checkout={end}"

# 6) Dynamic place resolution (city or IATA)
async def geocode_full(place: str) -> Optional[Dict[str, Any]]:
    import httpx
    async with httpx.AsyncClient(timeout=25) as client:
        r = await client.get("https://geocoding-api.open-meteo.com/v1/search", params={"name": place, "count": 1})
        data = r.json()
        res = data.get("results") or []
        if not res: return None
        g = res[0]
        return {"name": g.get("name") or place, "country": g.get("country",""), "lat": g["latitude"], "lon": g["longitude"]}

def looks_like_iata(s: str) -> bool:
    return isinstance(s, str) and len(s) == 3 and s.isalpha() and s.upper() == s

def _pick_city_from_text(text: str) -> Optional[str]:
    for pat in [
        r"serv(?:es|ing)\s+([A-Z][\w\- ]{2,40})",
        r"near\s+([A-Z][\w\- ]{2,40})",
        r"([A-Z][\w\- ]{2,40})\s+International Airport",
        r"in\s+([A-Z][\w\- ]{2,40})\s*(?:,|\.)",
    ]:
        m = re.search(pat, text)
        if m:
            cand = re.sub(r"[-–—|].*$", "", m.group(1)).strip()
            if 2 <= len(cand) <= 48: return cand
    return None

async def resolve_place(place: str) -> Tuple[str, Optional[Dict[str, Any]]]:
    g = await geocode_full(place)
    if g: return g["name"], g
    if looks_like_iata(place):
        try:
            from duckduckgo_search import DDGS
            with DDGS() as ddg:
                rs = list(ddg.text(f"{place} IATA airport city", max_results=6))
        except Exception:
            rs = []
        cands = []
        for r in rs:
            blob = f"{r.get('title','')} — {r.get('body','')}"
            cand = _pick_city_from_text(blob)
            if cand: cands.append(cand)
        for cand in cands:
            g2 = await geocode_full(cand)
            if g2: return g2["name"], g2
    return place, None

# 7) Agents (Planner & Researcher)
from pydantic_ai import Agent

PLANNER = Agent[None, PlanSkeleton](
    model=MODEL,
    mcp_servers=servers,
    system_prompt=(
        "You are the Planner. Given origin/destination city, dates (YYYY-MM-DD), budget and themes, "
        "produce a concise skeleton itinerary (3–10 days). "
        "Return PlanSkeleton {destination_city,start_date,end_date,day_by_day[]} with valid YYYY-MM-DD dates only."
    ),
    output_type=PlanSkeleton,
    retries=2, output_retries=2,
)

RESEARCHER = Agent[None, ResearchOutput](
    model=MODEL,
    mcp_servers=servers,
    system_prompt=(
        "You are the Researcher. For each day/date in the given city and themes, "
        "use web_search and fetch to find 2–3 specific things to do (landmark, cafe, hike). "
        "Return ResearchOutput with days[].ideas[] as Links(title,url,notes). Keep items relevant to the city."
    ),
    output_type=ResearchOutput,
    retries=2, output_retries=2,
)

# 8) Weather helpers
FORECAST_MAX_DAYS = 16

async def wx_forecast_exact(lat: float, lon: float, start_ymd: str, end_ymd: str, tz: str = "auto") -> List[Dict[str, Any]]:
    import httpx
    params = {
        "latitude": lat, "longitude": lon,
        "daily": ["temperature_2m_max","temperature_2m_min","precipitation_sum"],
        "timezone": tz, "start_date": start_ymd, "end_date": end_ymd,
    }
    try:
        async with httpx.AsyncClient(timeout=25) as client:
            d = (await client.get("https://api.open-meteo.com/v1/forecast", params=params)).json().get("daily", {})
    except Exception:
        return []
    times = d.get("time") or []; tmax = d.get("temperature_2m_max") or []; tmin = d.get("temperature_2m_min") or []; pmm = d.get("precipitation_sum") or []
    out = []
    for i, day in enumerate(times):
        try: out.append({"date": day, "tmax": float(tmax[i]), "tmin": float(tmin[i]), "precip_mm": float(pmm[i])})
        except Exception: pass
    return out

async def wx_forecast_window(lat: float, lon: float, need_days_from_today: int, tz: str = "auto") -> List[Dict[str, Any]]:
    import httpx
    params = {
        "latitude": lat, "longitude": lon,
        "daily": ["temperature_2m_max","temperature_2m_min","precipitation_sum"],
        "timezone": tz, "forecast_days": min(max(1, need_days_from_today), FORECAST_MAX_DAYS),
    }
    try:
        async with httpx.AsyncClient(timeout=25) as client:
            d = (await client.get("https://api.open-meteo.com/v1/forecast", params=params)).json().get("daily", {})
    except Exception:
        return []
    times = d.get("time") or []; tmax = d.get("temperature_2m_max") or []; tmin = d.get("temperature_2m_min") or []; pmm = d.get("precipitation_sum") or []
    out = []
    for i, day in enumerate(times):
        try: out.append({"date": day, "tmax": float(tmax[i]), "tmin": float(tmin[i]), "precip_mm": float(pmm[i])})
        except Exception: pass
    return out

async def wx_era5_last_year(lat: float, lon: float, start_ymd: str, end_ymd: str, tz: str = "auto") -> List[Dict[str, Any]]:
    import httpx
    ly_start = (datetime.strptime(start_ymd, "%Y-%m-%d") - timedelta(days=365)).strftime("%Y-%m-%d")
    ly_end   = (datetime.strptime(end_ymd,   "%Y-%m-%d") - timedelta(days=365)).strftime("%Y-%m-%d")
    params = {
        "latitude": lat, "longitude": lon,
        "daily": ["temperature_2m_max","temperature_2m_min","precipitation_sum"],
        "timezone": tz, "start_date": ly_start, "end_date": ly_end,
    }
    try:
        async with httpx.AsyncClient(timeout=25) as client:
            d = (await client.get("https://archive-api.open-meteo.com/v1/era5", params=params)).json().get("daily", {})
    except Exception:
        return []
    times = d.get("time") or []; tmax = d.get("temperature_2m_max") or []; tmin = d.get("temperature_2m_min") or []; pmm = d.get("precipitation_sum") or []
    out = []
    for i, day in enumerate(times):
        try: out.append({"date": day, "tmax": float(tmax[i]), "tmin": float(tmin[i]), "precip_mm": float(pmm[i])})
        except Exception: pass
    return out

def wx_tip(tmin: float, tmax: float, pmm: float) -> str:
    tip = "Light layers."
    if tmin < 12: tip = "Pack a light jacket."
    if pmm >= 3: tip += " Chance of rain — bring an umbrella."
    return f"Wx: {tmin:.0f}–{tmax:.0f}°C, {pmm:.0f} mm. {tip}"

async def build_weather_map(lat: float, lon: float, start_ymd: str, end_ymd: str) -> Dict[str, Dict[str, float]]:
    dates = daterange(start_ymd, end_ymd)
    wx = await wx_forecast_exact(lat, lon, start_ymd, end_ymd)
    wx_map = {r["date"]: r for r in wx}
    missing = [d for d in dates if d not in wx_map]
    if missing:
        today = datetime.utcnow().date()
        days_ahead = (datetime.strptime(start_ymd, "%Y-%m-%d").date() - today).days
        need_days = max(0, days_ahead) + len(dates)
        if days_ahead <= FORECAST_MAX_DAYS:
            window = await wx_forecast_window(lat, lon, need_days)
            for r in window:
                if r["date"] in dates:
                    wx_map[r["date"]] = r
            missing = [d for d in dates if d not in wx_map]
    if missing:
        ly = await wx_era5_last_year(lat, lon, start_ymd, end_ymd)
        ly_map = {r["date"]: r for r in ly}
        for d in missing:
            if d in ly_map: wx_map[d] = ly_map[d]
        missing = [d for d in dates if d not in wx_map]
    for d in missing:
        wx_map[d] = {"date": d, "tmax": 24.0, "tmin": 18.0, "precip_mm": 1.0}
    return wx_map

# 9) Deterministic cost model
def estimate_costs(distance_km: float, nights: int, days: int, budget_band: str = "economy") -> CostSummary:
    if distance_km <= 0:
        cpm = 0.10
    elif distance_km > 6000:
        cpm = 0.12
    elif distance_km > 3000:
        cpm = 0.10
    else:
        cpm = 0.07
    flights = distance_km * cpm

    band = (budget_band or "economy").lower()
    nightly = 140 if band == "economy" else (200 if band == "mid" else 280)
    hotels = nights * nightly

    act_day = 35 if band == "economy" else (50 if band == "mid" else 80)
    trn_day = 12 if band == "economy" else (18 if band == "mid" else 25)
    activities = days * act_day
    transit    = days * trn_day

    total = flights + hotels + activities + transit
    return CostSummary(currency="USD",
                       flights=round(flights, 2),
                       hotels=round(hotels, 2),
                       activities=round(activities, 2),
                       transit=round(transit, 2),
                       total=round(total, 2))

# 10) Orchestrator
from pydantic_ai import Agent

async def run_agent(agent: Agent, prompt: str):
    async with agent.run_stream(prompt) as stream:
        return await stream.get_output()

def _relevant_to_city(link: Link, city: str) -> bool:
    blob = f"{link.title or ''} {link.notes or ''} {link.url or ''}".lower()
    return city.lower() in blob

async def multi_agent_trip(origin_input: str, dest_input: str, start_date: str, end_date: str, budget: str, themes: str) -> TripPlan:
    # Resolve places
    origin_name, origin_geo = await resolve_place(origin_input)
    dest_name, dest_geo     = await resolve_place(dest_input)

    # 1) Planner
    planner_prompt = (
        f"Origin: {origin_name}\n"
        f"Destination: {dest_name}\n"
        f"Dates: {start_date} to {end_date}\n"
        f"Budget: {budget}\n"
        f"Themes: {themes}\n"
        "Return a PlanSkeleton."
    )
    print("🧭 Planner…")
    skel: PlanSkeleton = await run_agent(PLANNER, planner_prompt)

    # Dates: sanitize + fill all days
    safe_start = skel.start_date if _valid_ymd(skel.start_date) else start_date
    safe_end   = skel.end_date   if _valid_ymd(skel.end_date)   else end_date
    dates = daterange(safe_start, safe_end)  # inclusive
    day_set = set(d.date for d in (skel.day_by_day or []) if _valid_ymd(d.date))
    for d in dates:
        if d not in day_set:
            (skel.day_by_day).append(DayPlan(date=d, summary=f"{dest_name} highlights"))
    skel.day_by_day = sorted(skel.day_by_day, key=lambda x: x.date)

    # 2) Weather
    print("⛅ Weather…")
    if dest_geo:
        wx_map = await build_weather_map(dest_geo["lat"], dest_geo["lon"], safe_start, safe_end)
    else:
        wx_map = {d: {"date": d, "tmax": 24.0, "tmin": 18.0, "precip_mm": 1.0} for d in dates}

    # 3) Research (no fallback ideas; keep only city-relevant items)
    print("🔎 Research…")
    research_prompt = (
        f"City: {dest_name}\nDates: {', '.join(dates)}\nThemes: {themes}\n"
        "Return ResearchOutput with 2–3 ideas per date (Links with title,url,notes)."
    )
    research: ResearchOutput = await run_agent(RESEARCHER, research_prompt)
    research.days = [d for d in (research.days or []) if _valid_ymd(d.date)]
    ideas_map: Dict[str, List[Link]] = {}
    for d in research.days:
        filtered = [lnk for lnk in (d.ideas or []) if _relevant_to_city(lnk, dest_name)]
        ideas_map[d.date] = filtered[:3]

    # 4) Deterministic costs
    def haversine_km(lat1, lon1, lat2, lon2):
        if None in (lat1, lon1, lat2, lon2): return 0.0
        R = 6371.0
        p1, p2 = math.radians(lat1), math.radians(lat2)
        dphi = math.radians(lat2 - lat1)
        dlmb = math.radians(lon2 - lon1)
        a = math.sin(dphi/2)**2 + math.cos(p1)*math.cos(p2)*math.sin(dlmb/2)**2
        return 2 * R * math.asin(math.sqrt(a))
    lat1 = origin_geo["lat"] if origin_geo else None
    lon1 = origin_geo["lon"] if origin_geo else None
    lat2 = dest_geo["lat"]   if dest_geo   else None
    lon2 = dest_geo["lon"]   if dest_geo   else None
    distance_km = haversine_km(lat1, lon1, lat2, lon2)
    nights = max(0, (datetime.strptime(safe_end, "%Y-%m-%d") - datetime.strptime(safe_start, "%Y-%m-%d")).days)
    days = len(dates)
    costs = estimate_costs(distance_km, nights, days, budget)

    # 5) Links (Flights + Booking.com only) with computed prices
    per_night = (costs.hotels / max(1, nights)) if nights else None
    flights = [Link(
        title="Google Flights",
        url=google_flights_link(origin_name, dest_name, safe_start, safe_end),
        price=round(costs.flights, 0),
        notes="Estimated roundtrip (distance × CPM); click for live fares",
    )]
    hotels = [Link(
        title="Booking.com",
        url=booking_link(dest_name, safe_start, safe_end),
        price=round(costs.hotels, 0),
        notes=f"Estimated total for {nights} night(s)" + (f" (≈ ${per_night:,.0f}/night)" if per_night else ""),
    )]

    # 6) Merge day plans + weather tip
    merged_days: List[DayPlan] = []
    for d in skel.day_by_day:
        if not _valid_ymd(d.date): continue
        acts = list(d.activities or [])
        for link in ideas_map.get(d.date, []):
            acts.append(f"{link.title} — {link.url}" if link.url else link.title)
        w = wx_map.get(d.date)
        if w:
            acts.append(wx_tip(w["tmin"], w["tmax"], w["precip_mm"]))
        merged_days.append(DayPlan(date=d.date, summary=d.summary or f"{dest_name} highlights", activities=acts))

    # NOTE: Fixed Local transit text per request (always Taipei wording)
    fixed_transit_text = "Consider local transit passes in Taipei. Compare airport rail vs rideshare."

    return TripPlan(
        flights=flights,
        hotels=hotels,
        day_by_day=merged_days,
        transit_notes=fixed_transit_text,
        links=[],
        cost_summary=costs,
    )

def render_plan(plan: TripPlan) -> None:
    def _links_table(items: List[Link], label: str) -> str:
        if not items: return f"_No {label.lower()} returned._"
        rows = ["| # | Title | Price | Notes |", "|---:|---|---:|---|"]
        for i, l in enumerate(items, 1):
            title = f"[{l.title}]({l.url})" if l.url else (l.title or "")
            price = f"${l.price:,.0f}" if (l.price is not None) else ""
            notes = l.notes or ""
            rows.append(f"| {i} | {title} | {price} | {notes} |")
        return "\n".join(rows)

    def _day_table(days: List[DayPlan]) -> str:
        if not days: return "_No day-by-day plan returned._"
        rows = ["| Date | Plan |", "|---|---|"]
        for d in days:
            bullets = "<br>".join(f"• {a}" for a in (d.activities or []))
            plan = (d.summary or "") + (("<br>" + bullets) if bullets else "")
            rows.append(f"| {d.date} | {plan} |")
        return "\n".join(rows)

    parts = []
    parts += ["## ✈️ Flights", _links_table(plan.flights, "Flight links")]
    parts += ["\n\n## 🏨 Hotels", _links_table(plan.hotels, "Hotel links")]
    parts += ["\n\n## 📅 Day-by-day", _day_table(plan.day_by_day)]
    parts += ["\n\n## 🚉 Local transit", plan.transit_notes or "_—_"]
    if plan.links:
        parts += ["\n\n## 🔗 Extra links", _links_table(plan.links, "Links")]
    cs = plan.cost_summary
    parts += [
        "\n\n## 💵 Cost summary",
        f"""| Item | Cost ({cs.currency}) |
|---|---:|
| Flights | {cs.flights:,.0f} |
| Hotels | {cs.hotels:,.0f} |
| Activities | {cs.activities:,.0f} |
| Transit | {cs.transit:,.0f} |
| **Total** | **{cs.total:,.0f}** |"""
    ]
    display(Markdown("\n".join(parts)))

# 11) Demo helper
async def run_demo(
    origin_airport: str = "SFO",
    dest_city: str = "NRT",
    start_date: str = "2025-08-22",
    end_date: str = "2025-08-31",
    budget: str = "economy",
    themes: str = "Manga, sushi, japanese culture, and historical buildings",
):
    plan = await multi_agent_trip(origin_airport, dest_city, start_date, end_date, budget, themes)
    render_plan(plan)

print("🚀 Running demo...")
asyncio.run(run_demo())

Wrote servers to /sgl-workspace/hubert_sgl/mcp_nb
✗ time disabled: unhandled errors in a TaskGroup (1 sub-exception)
✗ search disabled: unhandled errors in a TaskGroup (1 sub-exception)
🚀 Running demo...
🧭 Planner…
[2025-08-15 18:59:50] INFO:     127.0.0.1:56394 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-08-15 18:59:50] Prefill batch. #new-seq: 1, #new-token: 56, #cached-token: 344, token usage: 0.02, #running-req: 0, #queue-req: 0, 
[2025-08-15 18:59:50] Decode batch. #running-req: 1, #token: 419, token usage: 0.02, cuda graph: True, gen throughput (token/s): 0.14, #queue-req: 0, 
[2025-08-15 18:59:50] Decode batch. #running-req: 1, #token: 459, token usage: 0.02, cuda graph: True, gen throughput (token/s): 186.93, #queue-req: 0, 
[2025-08-15 18:59:50] Decode batch. #running-req: 1, #token: 499, token usage: 0.02, cuda graph: True, gen throughput (token/s): 186.63, #queue-req: 0, 
[2025-08-15 18:59:50] Decode batch. #running-req: 1, #token: 539, token usage: 0.03, cuda graph:

🔎 Research…
[2025-08-15 18:59:54] INFO:     127.0.0.1:56394 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-08-15 18:59:54] Prefill batch. #new-seq: 1, #new-token: 162, #cached-token: 397, token usage: 0.02, #running-req: 0, #queue-req: 0, 
[2025-08-15 18:59:54] Decode batch. #running-req: 1, #token: 573, token usage: 0.03, cuda graph: True, gen throughput (token/s): 16.87, #queue-req: 0, 


## ✈️ Flights
| # | Title | Price | Notes |
|---:|---|---:|---|
| 1 | [Google Flights](https://www.google.com/travel/flights/search?q=Flights%20to%20Narita%20from%20San%20Francisco%20on%202025-08-22%20through%202025-08-31) | $986 | Estimated roundtrip (distance × CPM); click for live fares |


## 🏨 Hotels
| # | Title | Price | Notes |
|---:|---|---:|---|
| 1 | [Booking.com](https://www.booking.com/searchresults.html?ss=Narita&checkin=2025-08-22&checkout=2025-08-31) | $1,260 | Estimated total for 9 night(s) (≈ $140/night) |


## 📅 Day-by-day
| Date | Plan |
|---|---|
| 2025-08-22 | Arrival and accommodation setup<br>• Arrive at Narita Airport<br>• Check into hotel<br>• Set up accommodation<br>• Wx: 25–34°C, 0 mm. Light layers. |
| 2025-08-23 | Exploring Tokyo's Manga Culture<br>• Visit Akihabara for Manga and Anime<br>• Explore Tokyo Metropolitan Government Building<br>• Wx: 26–34°C, 0 mm. Light layers. |
| 2025-08-24 | Experience Traditional Japanese Culture<br>• Visit Tokyo National Museum<br>• Enjoy a traditional tea ceremony<br>• Wx: 22–31°C, 5 mm. Light layers. Chance of rain — bring an umbrella. |
| 2025-08-25 | Sushi and Japanese Cuisine<br>• Visit Tsukiji Outer Market for fresh sushi<br>• Dinner at a local izakaya<br>• Wx: 21–23°C, 4 mm. Light layers. Chance of rain — bring an umbrella. |
| 2025-08-26 | Historical Buildings and Temples<br>• Visit Senso-ji Temple<br>• Explore Asakusa district<br>• Wx: 21–27°C, 0 mm. Light layers. |
| 2025-08-27 | Shopping and Relaxation<br>• Visit Shibuya and Harajuku<br>• Relax at a local onsen<br>• Wx: 22–29°C, 0 mm. Light layers. |
| 2025-08-28 | Explore Tokyo’s Modern Side<br>• Visit Shinjuku Gyoen National Garden<br>• Shopping in Omotesando<br>• Wx: 25–34°C, 0 mm. Light layers. |
| 2025-08-29 | Day Trip to Yokohama<br>• Visit Yokohama Chinatown<br>• Yokohama Red Brick Warehouse<br>• Wx: 24–27°C, 1 mm. Light layers. |
| 2025-08-30 | Last Day Explorations<br>• Visit Tokyo Skytree<br>• Explore Odaiba area<br>• Wx: 22–25°C, 3 mm. Light layers. |
| 2025-08-31 | Departure from Narita Airport<br>• Pack and check out of hotel<br>• Departure from Narita Airport<br>• Wx: 18–24°C, 1 mm. Light layers. |


## 🚉 Local transit
Consider local transit passes in Taipei. Compare airport rail vs rideshare.


## 💵 Cost summary
| Item | Cost (USD) |
|---|---:|
| Flights | 986 |
| Hotels | 1,260 |
| Activities | 350 |
| Transit | 120 |
| **Total** | **2,716** |

<a id="step7"></a>

## Step 7: Challenge - Expand the Agent

**Task:** The challenge for this workshop will be announced during the workshop. 

In [62]:
# Example: uncomment any to try
# asyncio.run(run_demo("SFO", "China", "2025-08-22", "2025-08-31", "economy", "Food, Chinese culture, and historical buildings"))
asyncio.run(run_demo("SFO", "Seoul", "2025-08-22", "2025-07-02", "economy", "Food, Chinese culture, and historical buildings"))

🧭 Planner…
[2025-08-15 19:06:01] INFO:     127.0.0.1:43420 - "POST /v1/chat/completions HTTP/1.1" 200 OK
[2025-08-15 19:06:01] Prefill batch. #new-seq: 1, #new-token: 1, #cached-token: 396, token usage: 0.02, #running-req: 0, #queue-req: 0, 
[2025-08-15 19:06:01] Decode batch. #running-req: 1, #token: 434, token usage: 0.02, cuda graph: True, gen throughput (token/s): 0.75, #queue-req: 0, 
[2025-08-15 19:06:01] Decode batch. #running-req: 1, #token: 474, token usage: 0.02, cuda graph: True, gen throughput (token/s): 186.71, #queue-req: 0, 
[2025-08-15 19:06:01] Decode batch. #running-req: 1, #token: 514, token usage: 0.03, cuda graph: True, gen throughput (token/s): 186.42, #queue-req: 0, 
[2025-08-15 19:06:01] Decode batch. #running-req: 1, #token: 554, token usage: 0.03, cuda graph: True, gen throughput (token/s): 186.21, #queue-req: 0, 
[2025-08-15 19:06:02] Decode batch. #running-req: 1, #token: 594, token usage: 0.03, cuda graph: True, gen throughput (token/s): 186.06, #queue-req:

## ✈️ Flights
| # | Title | Price | Notes |
|---:|---|---:|---|
| 1 | [Google Flights](https://www.google.com/travel/flights/search?q=Flights%20to%20Seoul%20from%20San%20Francisco%20on%202025-08-22%20through%202025-08-02) | $1,083 | Estimated roundtrip (distance × CPM); click for live fares |


## 🏨 Hotels
| # | Title | Price | Notes |
|---:|---|---:|---|
| 1 | [Booking.com](https://www.booking.com/searchresults.html?ss=Seoul&checkin=2025-08-22&checkout=2025-08-02) | $0 | Estimated total for 0 night(s) |


## 📅 Day-by-day
| Date | Plan |
|---|---|
| 2025-08-22 | Welcome to Seoul, the vibrant heart of South Korea. Start your day with a visit to Gyeongbokgung Palace, a symbol of Korean history.<br>• Visit Gyeongbokgung Palace<br>• Explore Namsangol Hanok Village |
| 2025-08-23 | Today, we explore Seoul's cultural side. We'll visit the National Museum of Korea and The Seoul Museum of Art.<br>• Visit National Museum of Korea<br>• Explore The Seoul Museum of Art |
| 2025-08-24 | Sample traditional Korean cuisine at Myeong-dong. In the evening, enjoy a meal at a local restaurant in Bukchon Hanok Village.<br>• Lunch: Myeong-dong Street<br>• Dinner: Bukchon Hanok Village |
| 2025-08-25 | Discover the Chinese influence in Seoul with a visit to Manse Church and Chongmyo Royal Ancestral Shrine.<br>• Visit Manse Church<br>• Explore Chongmyo Royal Ancestral Shrine |
| 2025-08-26 | Explore the trendy neighborhoods of Hongdae and Itaewon. Enjoy a night out at Itaewon for shopping and dining.<br>• Visit Hongdae and Its Art Galleries<br>• Dine at Itaewon |
| 2025-08-27 | Day off to relax. Enjoy a spa or visit the nearby Insa-dong for shopping and tea houses.<br>• Relax at a Spa<br>• Visit Insa-dong |
| 2025-08-28 | Visit the modern side of Seoul with a trip to Lotte World Tower and visit the futuristic Dongdaemun Design Plaza.<br>• Visit Lotte World Tower<br>• Explore Dongdaemun Design Plaza |
| 2025-08-29 | Last day in Seoul! Visit the famous Myeong-dong shopping district and grab a last meal at a local restaurant.<br>• Shopping and sightseeing in Myeong-dong<br>• Dinner: Local Restaurant |


## 🚉 Local transit
Consider local transit passes in Taipei. Compare airport rail vs rideshare.


## 💵 Cost summary
| Item | Cost (USD) |
|---|---:|
| Flights | 1,083 |
| Hotels | 0 |
| Activities | 0 |
| Transit | 0 |
| **Total** | **1,083** |

Happy coding! If you encounter issues or have questions, don’t hesitate to ask or raise an issue on our [Github page](https://github.com/ROCm/gpuaidev)!